<a href="https://colab.research.google.com/github/nvk681/into_to_dl_hw6/blob/main/CODING/coding_assignment.ipynb#scrollTo=FaUQXs7AnYC6">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



In [1]:
# print(torch._version_)
# print(torchtext._version_)

In [2]:
# !pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html


In [3]:
# !pip install -U torch==1.8.0 torchtext==0.9.0

In [4]:
# pip install torchtext

In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd
import spacy

print(torch.__version__)
print(torchtext.__version__)


1.8.0+cu111
0.9.0


# Import Required Libraries & Data Loading

In [6]:
#importing the training data
df=pd.read_csv('IMDB Dataset.csv')
print(df.shape)
df.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


# Data Preparation

In [7]:
"""
sentiment : 0 = negative, 1 = positive 
use the following to get the sentiment of a sentence :  
sentiment = 0 if sentiment is negative else 1


use np.where to get the sentiment of a sentence :
"""
df['sentiment'] = np.where(df['sentiment'] == 'positive', 1, 0)

In [8]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [9]:
df.columns = ['TEXT_COLUMN_NAME', 'LABEL_COLUMN_NAME']

In [10]:
"""
Load the spacy model and load the English language model from https://spacy.io/usage/models
"""
# python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")
# spacy.### ADD YOUR SPACY MODEL HERE ###

In [11]:
# general Settings

RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.005 ### ADD YOUR LEARNING RATE HERE ###
BATCH_SIZE = 126### ADD YOUR BATCH SIZE HERE ###
NUM_EPOCHS = 10 ### ADD YOUR NUMBER OF EPOCHS HERE ###
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 400### ADD YOUR EMBEDDING DIMENSION HERE ###
HIDDEN_DIM = 128 ### ADD YOUR HIDDEN DIMENSION HERE ###
NUM_CLASSES = 2

In [12]:
# pip install torchtext

*italicized text*# Text & label Preparation

In [13]:
# pip install torch==1.0.0

In [14]:
# Define feature processing
"""
Define the fields for the data.
"""
# from torchtext import data
# TEXT = torchtext.data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm')
TEXT = torchtext.legacy.data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm')

In [15]:
# Define Label processing
LABEL = torchtext.legacy.data.LabelField(dtype = torch.long)

In [16]:
"""
Define the fields for the data.
"""

df.to_csv('moviedata.csv', index = None)
df = pd.read_csv('moviedata.csv')
df.head()

,TEXT_COLUMN_NAME,LABEL_COLUMN_NAME
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [17]:
# process the dataset
# TEXT = LABEL = []
fields = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]

dataset = torchtext.legacy.data.TabularDataset(
                    path = '/content/moviedata.csv', ### ADD YOUR DATASET PATH HERE ###
                    format = 'csv', ### ADD YOUR DATASET FORMAT HERE ###
                    skip_header = True, ### ADD YOUR SKIP HEADER HERE ### 
                    fields =  fields#("TEXT_COLUMN_NAME",	"LABEL_COLUMN_NAME")### ADD YOUR FIELDS HERE ### 
)

# Data Split

In [18]:
# Split dataset into train and test set

train_data, test_data = dataset.split(split_ratio = [0.8, 0.2], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of test data', len(test_data))

Length of train data 40000
Length of test data 10000


In [19]:
train_data, val_data = train_data.split(split_ratio = [0.85, 0.15], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of valid data', len(val_data))

Length of train data 34000
Length of valid data 6000


# Data Observation after Tokenization

In [20]:
# Look at first traning example

print(vars(train_data.examples[2000]))

{'TEXT_COLUMN_NAME': ['Flipping', 'through', 'the', 'channels', 'I', 'was', 'lucky', 'enough', 'to', 'stumble', 'upon', 'the', 'beginning', 'of', 'this', 'movie', '.', 'I', 'must', 'admit', 'that', 'it', 'grabbed', 'my', 'attention', 'almost', 'immediately', '.', 'I', 'love', 'older', 'films', 'and', 'this', 'is', 'or', 'should', 'be', 'considered', 'a', 'classic', '!', 'One', 'of', 'the', 'most', 'wonderful', 'rarities', 'of', 'this', 'movie', 'is', 'that', 'the', 'main', 'character', 'was', 'not', 'only', 'female', 'but', 'she', 'was', 'also', 'a', 'bad', 'girl', '.', 'I', 'highly', 'recommend', 'this', 'movie', '!'], 'LABEL_COLUMN_NAME': '1'}


In [21]:
# Build Vocabulary

TEXT.build_vocab(train_data, max_size = VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'vocabulary size: {len(TEXT.vocab)}')
print(f'Label Size: {len(LABEL.vocab)}')

vocabulary size: 20002
Label Size: 2


 2 extra value in vocabulary is because added (unknown) and (padding)

In [22]:
# Print the most common words: Use the most_common method of the TEXT vocabulary
# import collections

most_common_words = TEXT.vocab.freqs.most_common(10)
print(most_common_words)

[('the', 390972), (',', 369444), ('.', 318719), ('a', 210502), ('and', 210006), ('of', 194658), ('to', 180163), ('is', 145895), ('in', 118266), ('I', 105681)]


In [23]:
# Token corresponding to first 10 Indices

print(TEXT.vocab.itos[:20]) #itos = Integer to string

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is', 'in', 'I', 'it', 'that', '"', "'s", 'this', '-', '/><br', 'was']


# Data Preparation for Batch wise Implimentation

In [24]:
# Define Dataloader
    # sort = False, #don't sort test/validation data
    # batch_size=BATCH_SIZE,
    # device=device)
train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
        (train_data, val_data, test_data), ### ADD YOUR SPLIT DATA HERE (Make sure you add it in a tuple) ###
        batch_size = 128, ### ADD YOUR BATCH SIZE HERE ###
        sort_within_batch = False, ### ADD YOUR SORT WITHIN BATCH HERE ### 
        sort_key = lambda x : len(x.TEXT_COLUMN_NAME), 
        #device = DEVICE
    )

In [25]:
# Testing the iterators (note that the number of rows depends on the longest document in the respective batch):

print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

Train
Text matrix size: torch.Size([1154, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([56, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([50, 128])
Target vector size: torch.Size([128])


In [26]:
# train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
#         , ### ADD YOUR SPLIT DATA HERE (Make sure you add it in a tuple) ###
#         batch_size = 64, ### ADD YOUR BATCH SIZE HERE ###
#         sort_within_batch = Flase, ### ADD YOUR SORT WITHIN BATCH HERE ###
#         sort_key = lambda x : len(x.TEXT_COLUMN_NAME),
#         device = DEVICE
#     )

# Model Building

In [27]:
import torch.nn as nn

class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        ### ADD YOUR CODE HERE ###
        # super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        ### END YOUR CODE ### 

    def forward(self, text):
        ### ADD YOUR CODE HERE ###
        # text dim: [sentence length, batch size]
        
        # embedded dim: [sentence length, batch size, embedding dim]
        
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]

        # hidden dim: [batch size, hidden dim]

        
        ### END YOUR CODE ###
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze_(0))

In [28]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim=20002, ### ADD YOUR INPUT DIM HERE. This can be the length of your vocabulary or the embedding dim ###
            embedding_dim=EMBEDDING_DIM, ### ADD YOUR EMBEDDING DIM HERE ###
            hidden_dim=HIDDEN_DIM, ### ADD YOUR HIDDEN DIM HERE ###
            output_dim=2  ### ADD NUMBER OF CLASSES HERE ###
)

model = model.to(DEVICE)
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-3)
# optimizer = optim.SGD(model.parameters(), lr=1e-3)


# Define Accuracy

In [29]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

# Model Run

In [30]:
start_time = time.time()
# criterion = nn.functional.cross_entropy()
# criterion = criterion.to(DEVICE)

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)

        ### FORWARD AND BACK PROP
        predictions = model(text)#.squeeze(1)

                
        # print(labels)
        # print(predictions)
        # predictions_scaled = predictions.squeeze(1)#igmoid = torch.round(torch.sigmoid(predictions))
        # print(predictions_scaled)

        loss = nn.functional.cross_entropy(predictions, labels)#.float())#criterion(predictions_scaled, labels)
        
        # acc = compute_accuracy(predictions, labels, DEVICE)
        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        
        
        
        ### UPDATE MODEL PARAMETERS
        
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/010 | Batch 000/266 | Loss: 0.8814
Epoch: 001/010 | Batch 050/266 | Loss: 0.6994
Epoch: 001/010 | Batch 100/266 | Loss: 0.6899
Epoch: 001/010 | Batch 150/266 | Loss: 0.6959
Epoch: 001/010 | Batch 200/266 | Loss: 0.6990
Epoch: 001/010 | Batch 250/266 | Loss: 0.6928
training accuracy: 49.94%
valid accuracy: 50.02%
Time elapsed: 0.50 min
Epoch: 002/010 | Batch 000/266 | Loss: 0.6997
Epoch: 002/010 | Batch 050/266 | Loss: 0.6899
Epoch: 002/010 | Batch 100/266 | Loss: 0.6855
Epoch: 002/010 | Batch 150/266 | Loss: 0.6988
Epoch: 002/010 | Batch 200/266 | Loss: 0.7140
Epoch: 002/010 | Batch 250/266 | Loss: 0.6993
training accuracy: 50.01%
valid accuracy: 50.12%
Time elapsed: 1.00 min
Epoch: 003/010 | Batch 000/266 | Loss: 0.6923
Epoch: 003/010 | Batch 050/266 | Loss: 0.6897
Epoch: 003/010 | Batch 100/266 | Loss: 0.6981
Epoch: 003/010 | Batch 150/266 | Loss: 0.6958
Epoch: 003/010 | Batch 200/266 | Loss: 0.6956
Epoch: 003/010 | Batch 250/266 | Loss: 0.6935
training accuracy: 50.31%
va

# Model Testing

In [31]:
import spacy


nlp = spacy.blank("en")

def predict_sentiment(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    return prediction[0][1].item()

print('Probability positive:')
predict_sentiment(model, "This is such an awesome movie, I really love it!")

Probability positive:


0.19686156511306763

In [32]:
print('Probability positive:')
predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")

Probability positive:


0.0539405457675457